In [ ]:
import pandas as pd
# import matplotlib.pyplot as plt
# import seaborn as sns
# import numpy as np
# import nltk

In [ ]:
!pip install datasets

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 510.5/510.5 kB 9.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 116.3/116.3 kB 11.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 194.1/194.1 kB 15.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 134.8/134.8 kB 14.0 MB/s eta 0:00:00


In [ ]:
!pip install transformers

In [ ]:
!pip install transformers[torch]

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 297.6/297.6 kB 5.7 MB/s eta 0:00:00
  Using cached nvidia_cuda_nvrtc_cu12-12.1.105-py3-none-manylinux1_x86_64.whl (23.7 MB)
  Using cached nvidia_cuda_runtime_cu12-12.1.105-py3-none-manylinux1_x86_64.whl (823 kB)
  Using cached nvidia_cuda_cupti_cu12-12.1.105-py3-none-manylinux1_x86_64.whl (14.1 MB)
  Using cached nvidia_cudnn_cu12-8.9.2.26-py3-none-manylinux1_x86_64.whl (731.7 MB)
  Using cached nvidia_cublas_cu12-12.1.3.1-py3-none-manylinux1_x86_64.whl (410.6 MB)
  Using cached nvidia_cufft_cu12-11.0.2.54-py3-none-manylinux1_x86_64.whl (121.6 MB)
  Using cached nvidia_curand_cu12-10.3.2.106-py3-none-manylinux1_x86_64.whl (56.5 MB)
  Using cached nvidia_cusolver_cu12-11.4.5.107-py3-none-manylinux1_x86_64.whl (124.2 MB)
  Using cached nvidia_cusparse_cu12-12.1.0.106-py3-none-manylinux1_x86_64.whl (196.0 MB)
  Using cached nvidia_nccl_cu12-2.19.3-py3-none-manylinux1_x86_64.whl (166.0 MB)
  Using cached nvidia_nvtx_cu12-12.1.105-py3-none-many

In [ ]:
!pip install accelerate

In [ ]:
!pip install rouge_score

  Preparing metadata (setup.py) ... done
  Created wheel for rouge_score: filename=rouge_score-0.1.2-py3-none-any.whl size=24933 sha256=d0cd2e255358f04165c01990416031029843229eed5f7293f250c9954ecbe938
  Stored in directory: /root/.cache/pip/wheels/5f/dd/89/461065a73be61a532ff8599a28e9beef17985c9e9c31e541b4
Successfully built rouge_score


In [ ]:
!pip install evaluate

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 84.1/84.1 kB 2.7 MB/s eta 0:00:00


In [ ]:
# import transformers
from transformers import BartTokenizer, BartForConditionalGeneration, Trainer, TrainingArguments, Seq2SeqTrainingArguments
from datasets import load_dataset, load_from_disk, Dataset
from sklearn.model_selection import train_test_split

ModuleNotFoundError: No module named 'datasets'

In [ ]:
import accelerate

import os
os.environ["PYTORCH_CUDA_ALLOC_CONF"] = "expandable_segments:True"

import torch
# your PyTorch code

In [ ]:
# import transformers
# import accelerate
# import torch

# print("Transformers version:", transformers.__version__)
# print("Accelerate version:", accelerate.__version__)
# print("PyTorch version:", torch.__version__)

In [ ]:
from google.colab import drive
drive.mount('/content/gdrive')

Mounted at /content/gdrive


In [ ]:
data = pd.read_csv("gdrive/My Drive/recipe_dataset.csv")

In [ ]:
data.shape
# print(data['answer'][0])

(1000, 2)

In [ ]:
df = pd.DataFrame(data)
df_random_8000 = df.sample(n=500, random_state=42)
df = df_random_8000
train_df, test_df = train_test_split(df, test_size=0.2)
train_dataset = Dataset.from_pandas(train_df)
test_dataset = Dataset.from_pandas(test_df)

In [ ]:
train_dataset.shape

(400, 3)

In [ ]:
test_dataset.shape

(100, 3)

In [ ]:
# max_input = 512
# max_target = 128
# batch_size = 3
max_length = 512
tokenizer = BartTokenizer.from_pretrained('facebook/bart-large')
model = BartForConditionalGeneration.from_pretrained('facebook/bart-large')

/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_token.py:88: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


tokenizer_config.json:   0%|          | 0.00/26.0 [00:00<?, ?B/s]

vocab.json:   0%|          | 0.00/899k [00:00<?, ?B/s]

merges.txt:   0%|          | 0.00/456k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/1.36M [00:00<?, ?B/s]

config.json:   0%|          | 0.00/1.63k [00:00<?, ?B/s]

pytorch_model.bin:   0%|          | 0.00/1.02G [00:00<?, ?B/s]

In [ ]:
def preprocess_function(examples):
    inputs = [q for q in examples['question']]
    model_inputs = tokenizer(inputs,padding='max_length', max_length=max_length, truncation=True)

    with tokenizer.as_target_tokenizer():
        labels = tokenizer(examples['answer'], padding='max_length', max_length=max_length, truncation=True)

    model_inputs["labels"] = labels["input_ids"]
    return model_inputs

train_tokenized = train_dataset.map(preprocess_function, batched=True)
test_tokenized = test_dataset.map(preprocess_function, batched=True)

Map:   0%|          | 0/400 [00:00<?, ? examples/s]

/usr/local/lib/python3.10/dist-packages/transformers/tokenization_utils_base.py:3892: UserWarning: `as_target_tokenizer` is deprecated and will be removed in v5 of Transformers. You can tokenize your labels by using the argument `text_target` of the regular `__call__` method (either in the same call as your input texts if you use the same keyword arguments, or in a separate call.
  warnings.warn(


Map:   0%|          | 0/100 [00:00<?, ? examples/s]

In [ ]:
# from transformers import Trainer, TrainingArguments
from transformers import Seq2SeqTrainingArguments, Seq2SeqTrainer


training_args = Seq2SeqTrainingArguments(
    output_dir='./results',          # output directory
    num_train_epochs=3,              # number of training epochs
    per_device_train_batch_size=2,   # batch size for training
    per_device_eval_batch_size=2,    # batch size for evaluation
    warmup_steps=500,                # number of warmup steps for learning rate scheduler
    weight_decay=0.01,               # strength of weight decay
    logging_dir='./logs',            # directory for storing logs
    logging_steps=10,
    gradient_accumulation_steps=8,
    learning_rate=3e-4,
)

In [ ]:
import nltk, evaluate
import numpy as np
from nltk.tokenize import sent_tokenize
nltk.download("punkt", quiet=True)
metric = evaluate.load("rouge")

def compute_metrics(eval_preds):
    preds, labels = eval_preds

    # decode preds and labels
    labels = np.where(labels != -100, labels, tokenizer.pad_token_id)
    decoded_preds = tokenizer.batch_decode(preds, skip_special_tokens=True)
    decoded_labels = tokenizer.batch_decode(labels, skip_special_tokens=True)

    # rougeLSum expects newline after each sentence
    decoded_preds = ["\n".join(nltk.sent_tokenize(pred.strip())) for pred in decoded_preds]
    decoded_labels = ["\n".join(nltk.sent_tokenize(label.strip())) for label in decoded_labels]

    result = metric.compute(predictions=decoded_preds, references=decoded_labels, use_stemmer=True)
    return result


In [ ]:
trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=train_tokenized,
    eval_dataset=test_tokenized,
    tokenizer=tokenizer,
    compute_metrics=metric
)

/usr/local/lib/python3.10/dist-packages/accelerate/accelerator.py:436: FutureWarning: Passing the following arguments to `Accelerator` is deprecated and will be removed in version 1.0 of Accelerate: dict_keys(['dispatch_batches', 'split_batches', 'even_batches', 'use_seedable_sampler']). Please pass an `accelerate.DataLoaderConfiguration` instead: 
dataloader_config = DataLoaderConfiguration(dispatch_batches=None, split_batches=False, even_batches=True, use_seedable_sampler=True)
  warnings.warn(


In [ ]:
trainer.train()

Step,Training Loss
10,17.523600
20,12.921800
30,11.083600
40,9.754100
50,7.888600
60,6.132600
70,5.220600


TrainOutput(global_step=75, training_loss=9.704996999104818, metrics={'train_runtime': 523.9437, 'train_samples_per_second': 2.29, 'train_steps_per_second': 0.143, 'total_flos': 1300262761267200.0, 'train_loss': 9.704996999104818, 'epoch': 3.0})

In [ ]:
trainer.evaluate()

In [ ]:
trainer.save_model ("gdrive/My Drive/bart500")

Some non-default generation parameters are set in the model config. These should go into a GenerationConfig file (https://huggingface.co/docs/transformers/generation_strategies#save-a-custom-decoding-strategy-with-your-model) instead. This warning will be raised to an exception in v4.41.
Non-default generation parameters: {'early_stopping': True, 'num_beams': 4, 'no_repeat_ngram_size': 3, 'forced_bos_token_id': 0, 'forced_eos_token_id': 2}


In [ ]:
# from transformers import BartForConditionalGeneration, BartTokenizer

# model = BartForConditionalGeneration.from_pretrained('./saved_model/bart8000')
# tokenizer = BartTokenizer.from_pretrained('./saved_model/bart8000')
model = BartForConditionalGeneration.from_pretrained("gdrive/My Drive/bart500", local_files_only=True)

# Encode input context
input_text = "What dishes can we make with potato, salmon, and Broccoli"
inputs = tokenizer(input_text, return_tensors="pt")

# # Generate a response
# outputs = model.generate(inputs['input_ids'], num_beams=5, early_stopping=True)
# print(tokenizer.decode(outputs[0], skip_special_tokens=True))

# Debug: Check input tokens
# print("Tokenized inputs:", tokenizer.convert_ids_to_tokens(inputs['input_ids'][0]))

outputs = model.generate(**inputs, max_length=512)
generated_text = tokenizer.decode(outputs[0], skip_special_tokens=True)

# Debug: Check generated text
print(tokenizer.batch_decode(outputs, skip_special_tokens=True))


NameError: name 'BartForConditionalGeneration' is not defined

In [ ]:
# Encode input context
input_text = "What dishes can we make with bread , butter"
inputs = tokenizer(input_text, return_tensors="pt")

# # Generate a response
# outputs = model.generate(inputs['input_ids'], num_beams=5, early_stopping=True)
# print(tokenizer.decode(outputs[0], skip_special_tokens=True))

# Debug: Check input tokens
# print("Tokenized inputs:", tokenizer.convert_ids_to_tokens(inputs['input_ids'][0]))

outputs = model.generate(**inputs, max_length=512)
generated_text = tokenizer.decode(outputs[0], skip_special_tokens=True)

# Debug: Check generated text
print(tokenizer.batch_decode(outputs, skip_special_tokens=True))

['You can make Sweet Potato Pie.  Here is the recipe : Mix all ingredients together.  Bake at 350° for 30 minutes. ']


In [ ]:
# Encode input context
input_text = "What dishes can we make with cucumber , egg"
inputs = tokenizer(input_text, return_tensors="pt")

# # Generate a response
# outputs = model.generate(inputs['input_ids'], num_beams=5, early_stopping=True)
# print(tokenizer.decode(outputs[0], skip_special_tokens=True))

# Debug: Check input tokens
# print("Tokenized inputs:", tokenizer.convert_ids_to_tokens(inputs['input_ids'][0]))

outputs = model.generate(**inputs, max_length=512)
generated_text = tokenizer.decode(outputs[0], skip_special_tokens=True)

# Debug: Check generated text
print(tokenizer.batch_decode(outputs, skip_special_tokens=True))


['You can make Egg Salad.  Here is the recipe : Mix all ingredients together in a large bowl.  Place in a greased casserole dish.  Bake at 350° for 30 minutes. ']
